# Testing the old code

In [6]:

import os
# Stopping Tensorflow from printing info messages and warnings.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import comb
from itertools import combinations_with_replacement
import tensorflow as tf
import logging
import time
import random
import sys